# Import dependencies

In [2]:
if 'Imports':
    # from Scripts.DataManager.DabasePreparations.AmazonReviewSentiGraph import AmazonReviewSentiGraph
    from Scripts.Models.ModelsManager.SimpleGraphClassifierModelManager import SimpleGraphClassifierModelManager
    from Scripts.Configs.ConfigClass import Config
    config = Config(r'C:\Users\fardin\Projects\ColorIntelligence')
    from Scripts.DataManager.GraphConstructor.GraphConstructor import TextGraphType

C:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


# Load and prepare data

In [3]:
from os import path

import pandas as pd
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader

from Scripts.Configs.ConfigClass import Config
from Scripts.DataManager.GraphConstructor.CoOccurrenceGraphConstructor import CoOccurrenceGraphConstructor
from Scripts.DataManager.GraphConstructor.GraphConstructor import GraphConstructor, TextGraphType
from Scripts.DataManager.GraphLoader.GraphLoader import GraphLoader
from torch.utils.data.dataset import random_split
import torch
from Scripts.DataManager.Datasets.GraphConstructorDataset import GraphConstructorDataset

class AmazonReviewGraphLoader(GraphLoader):

    def __init__(self, config: Config, has_val: bool, has_test: bool, test_size=0.2, val_size=0.2, num_workers=2,
                 drop_last=True, train_data_path='', test_data_path='', graphs_path='', batch_size = 32,
                 device='cpu', shuffle = False, num_data_load=-1,
                 graph_type: TextGraphType = TextGraphType.CO_OCCURRENCE, *args, **kwargs):
        kwargs['num_workers'] = num_workers
        kwargs['batch_size'] = batch_size
        kwargs['num_workers'] = num_workers
        kwargs['shuffle'] = shuffle
        super(AmazonReviewGraphLoader, self)\
            .__init__(config, device, has_val, has_test, test_size, val_size, *args, **kwargs)

        self.device = device
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.drop_last = drop_last
        self.graph_type = graph_type
        self.train_data_path = 'data/Amazon-Review/train_sm.csv' if train_data_path == '' else train_data_path
        self.test_data_path = 'data/Amazon-Review/test_sm.csv' if test_data_path == '' else test_data_path
        self.train_df: pd.DataFrame = pd.DataFrame()
        self.test_df: pd.DataFrame = pd.DataFrame()
        self.labels = None
        self.dataset = None
        self.shuffle = shuffle
        self.num_node_features = 0
        self.num_classes = 0
        self.df: pd.DataFrame = pd.DataFrame()
        self.__train_dataset, self.__val_dataset, self.__test_dataset = None, None, None
        self.num_data_load = num_data_load

    def prepare_data(self):
        pass

    def setup(self, stage: str):
        pass
    def setup2(self):
        self.train_df = pd.read_csv(path.join(self.config.root, self.train_data_path))
        self.test_df = pd.read_csv(path.join(self.config.root, self.test_data_path))
        self.train_df.columns = ['Polarity', 'Title', 'Review']
        self.test_df.columns = ['Polarity', 'Title', 'Review']
        self.train_df = self.train_df[['Polarity', 'Review']]
        self.test_df = self.test_df[['Polarity', 'Review']]
        self.df = pd.concat([self.train_df, self.test_df])
        labels = self.df['Polarity']
        self.num_data_load = self.num_data_load if self.num_data_load>0 else self.df.shape[0]
        labels = labels.apply(lambda p: 0 if p == 1 else 1).to_numpy()[:self.num_data_load]
        labels = torch.from_numpy(labels)
        self.labels = labels.to(torch.float32).view(-1, 1)
        graph_constructor = self.__get_co_occurrence_graph()
        self.dataset = GraphConstructorDataset(graph_constructor, self.labels)
        sample_graph = graph_constructor.get_first()
        self.num_node_features = sample_graph.num_features
        self.num_classes = len(torch.unique(self.labels))
        self.__train_dataset, self.__val_dataset, self.__test_dataset =\
            random_split(self.dataset, [1-self.val_size-self.test_size, self.val_size, self.test_size])

    def teardown(self, stage: str) -> None:
        pass

    def train_dataloader(self):
        print(self.__train_dataset)
        print(self.batch_size)
        print(self.drop_last)
        print(self.shuffle)
        print(self.num_workers)
        return DataLoader(self.__train_dataset, batch_size=self.batch_size, drop_last=self.drop_last,
                          shuffle=self.shuffle, num_workers=self.num_workers, persistent_workers=True)

    def test_dataloader(self):
        return DataLoader(self.__test_dataset, batch_size=self.batch_size,
                          num_workers=self.num_workers, persistent_workers=True)

    def val_dataloader(self):
        return DataLoader(self.__val_dataset, batch_size=self.batch_size,
                          num_workers=self.num_workers, persistent_workers=True)

    def __set_graph_constructors(self, graph_type: TextGraphType):
        graph_constructors = {}
        if TextGraphType.CO_OCCURRENCE in graph_type:
            graph_constructors[TextGraphType.CO_OCCURRENCE] = self.__get_co_occurrence_graph()
        if TextGraphType.DEPENDENCY in graph_type:
            pass
        if TextGraphType.SEQUENTIAL in graph_type:
            pass
        if TextGraphType.TAGS in graph_type:
            pass
        return graph_constructors

    def __get_co_occurrence_graph(self):
        return CoOccurrenceGraphConstructor(self.train_df['Review'][:self.num_data_load], 'data/GraphData/AmazonReview', self.config,
                                         lazy_construction=False,
                                         load_preprocessed_data=False, naming_prepend='graph')

In [4]:
# from Scripts.DataManager.GraphLoader.AmazonReviewGraphLoader import AmazonReviewGraphLoader

In [31]:
if 'Load and Prepare data':
    data_manager = AmazonReviewGraphLoader(config, True, True, num_workers=2, batch_size=8, shuffle=True, num_data_load = -1)

C:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\data\lightning\datamodule.py:49: The 'shuffle=True' option is ignored in 'AmazonReviewGraphLoader'. Remove it from the argument list to disable this warning


# Train Data

In [32]:
data_manager.setup2()
data_loader = data_manager.train_dataloader()

i: 0
i: 100
i: 200
i: 300
i: 400
i: 500
i: 600
i: 700
i: 800
i: 900
i: 1000
i: 1100
i: 1200
i: 1300
i: 1400
i: 1500
i: 1600
i: 1700
i: 1800
i: 1900
i: 2000
i: 2100
i: 2200
i: 2300
i: 2400
i: 2500
i: 2600
i: 2700
i: 2800
i: 2900
i: 3000
i: 3100
i: 3200
i: 3300
i: 3400
i: 3500
i: 3600
i: 3700
i: 3800
i: 3900
i: 4000
i: 4100
i: 4200
i: 4300


KeyboardInterrupt: 

In [17]:
print(data_loader.dataset.__len__())

36


In [18]:
len(data_loader)

4

In [19]:
import time


In [20]:
begin = time.time()
for X,y in data_loader:
    print(f'{len(X)}, exectime: {time.time()-begin}')
    begin = time.time()

8, exectime: 37.84641790390015
8, exectime: 0.3111612796783447
8, exectime: 0.0010538101196289062
8, exectime: 0.0029754638671875


In [21]:
from Scripts.Models.BaseModels.GcnGatModel1 import GcnGatModel1
from Scripts.Models.LightningModels.LightningModels import BinaryLightningModel
import torch
import lightning as L

In [22]:
torch_model = GcnGatModel1(300, 1)
lightning_model = BinaryLightningModel(torch_model,
                                 torch.optim.Adam(torch_model.parameters(), lr=0.001, weight_decay=0.005),
                                       torch.nn.BCEWithLogitsLoss())

In [29]:
isinstance(data_manager, L.LightningDataModule)

False

In [30]:
trainer = L.Trainer(max_epochs=10, accelerator='gpu', devices=1, num_sanity_val_steps=0)
# trainer.fit(lightning_model, datamodule=data_manager)
trainer.fit(lightning_model,
            train_dataloaders=data_manager.train_dataloader(),
            val_dataloaders=data_manager.val_dataloader())

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | GcnGatModel1      | 245 K 
1 | loss_func | BCEWithLogitsLoss | 0     
2 | train_acc | BinaryAccuracy    | 0     
3 | val_acc   | BinaryAccuracy    | 0     
4 | test_acc  | BinaryAccuracy    | 0     
------------------------------------------------
245 K     Trainable params
0         Non-trainable params
245 K     Total params
0.983     Total estimated model params size (MB)


8
True
True
2


C:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\loops\fit_loop.py:293: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import summary
from tqdm import tqdm
from Scripts.Models.LightningModels.LightningModels import BaseLightningModel
from Scripts.Models.ModelsManager.ModelManager import ModelManager
from Scripts.Models.BaseModels.GATGCNClassifierSimple import GNNClassifier
from Scripts.DataManager.GraphLoader.NLabeledGraphLoader import NLabeledGraphLoader
from Scripts.Utils.enums import Optimizer, LossType

import lightning as L


class SimpleNodeClassifierModelManager(ModelManager):

    def __init__(self, graph_handler: NLabeledGraphLoader, device=torch.device('cpu'),
                 lr=0.01, weight_decay=0.001, optimizer_type: Optimizer = Optimizer.ADAM,
                 loss_type: LossType = LossType.CROSS_ENTROPY):
        super(SimpleNodeClassifierModelManager, self).__init__(lr, weight_decay, device)
        self.graph_handler = graph_handler
        self.num_output_classes = self.graph_handler.num_classes
        self.num_input_features = self.graph_handler.num_features
        self.loss_type = loss_type
        self.optimizer_type = optimizer_type
        self.model, self.optimizer, self.loss_func = self._create_model(lr, weight_decay, optimizer_type, loss_type)
        self.lightning_model = BaseLightningModel(self.model, self.optimizer, self.loss_func)

    def train(self, epoch_num: int = 100, lr: float = None, l2_norm: float = None, optimizer: Optimizer = None):

        trainer = L.Trainer(max_epochs=100, accelerator='gpu', devices=1)
        trainer.fit(self.lightning_model,
                    train_dataloaders=self.graph_handler.get_train_data(),
                    val_dataloaders=self.graph_handler.get_val_data()
                    )



        if lr or l2_norm or optimizer:
            self.set_optimizer(lr, l2_norm, optimizer)

        train_losses = []
        train_accuracies = []
        test_losses = []
        test_accuracies = []

        train_node_x, train_node_y, train_edges = self.graph_handler.get_train_data()
        test_node_x, test_node_y, test_edges = self.graph_handler.get_test_data()
        for i in tqdm(range(epoch_num)):
            self.model.train()
            self.optimizer.zero_grad()
            my_node, my_label, my_edges = self.graph_handler.extract_random_sub_edges_graph()
            y_hat: torch.Tensor = self.model(my_node, my_edges)
            loss = self.loss_func(F.one_hot(my_label, self.num_output_classes).float(), y_hat)
            loss.backward()
            self.optimizer.step()

            self.model.eval()

            loss_value = loss.item()
            train_losses.append(loss_value)
            accuracy = (torch.sum((y_hat.argmax(dim=1) == my_label).float()) / len(my_label)).cpu().numpy()
            train_accuracies.append(accuracy)

            my_node, my_label, my_edges = self.graph_handler.extract_random_sub_edges_graph()
            y_hat: torch.Tensor = self.model(my_node, my_edges)
            loss = self.loss_func(F.one_hot(my_label, self.graph_handler.num_classes).float(), y_hat)
            loss_value = loss.item()
            test_losses.append(loss_value)
            accuracy = (torch.sum((y_hat.argmax(dim=1) == my_label).float()) / len(my_label)).cpu().numpy()
            test_accuracies.append(accuracy)
        self.history['train_losses'] = train_losses
        self.history['train_accuracies'] = train_accuracies
        self.history['test_losses'] = test_losses
        self.history['test_accuracies'] = test_accuracies
        return self.history

    def evaluate(self):
        node_x, node_y, edges = self.graph_handler.get_val_data()
        self.model.eval()
        y_hat: torch.Tensor = self.model(node_x, edges)
        loss = self.loss_func(F.one_hot(node_y, self.num_output_classes).float(), y_hat)
        return y_hat, loss

    def predict(self, node_x, edge_index):
        self.model.eval()
        y_hat: torch.Tensor = self.model(node_x, edge_index)
        return y_hat

    def draw_summary(self):
        nodes_x, nodes_y, edge_indices_test = self.graph_handler.get_test_data()
        nodes_x, nodes_y, edge_indices_test = self.graph_handler.extract_random_sub_edges_graph(2)
        print(summary(self.model, nodes_x, edge_indices_test))

    def _create_model(self, lr, l2_norm, optimizer_type, loss_type):
        model = GNNClassifier(input_feature=self.num_input_features, class_counts=self.num_output_classes)
        optimizer = ModelManager._create_optimizer(model, lr, l2_norm, optimizer_type)
        loss_func = ModelManager._create_loss_func(loss_type)
        model.to(self.device)
        return model, optimizer, loss_func

    def set_optimizer(self, lr, l2_norm, optimizer=Optimizer.ADAM):
        if lr:
            self.lr = lr
        if l2_norm:
            self.l2_norm = l2_norm
        if optimizer:
            self.optimizer_type = optimizer
        self.optimizer = ModelManager._create_optimizer(self.model, self.lr, self.l2_norm, self.optimizer_type)


In [ ]:
# from Scripts.Models.ModelsManager.SimpleGraphClassifierModelManager import SimpleGraphClassifierModelManager

if 'Train Data':
    model_manager = SimpleGraphClassifierModelManager(
        data_manager.graph_constructors[TextGraphType.CO_OCCURRENCE])
    model_manager.draw_summary()
    model_manager.train()
    model_manager.draw_training_results()

# Draw Results

In [ ]:
from Scripts.Models.BaseModels.GcnGatModel1 import GcnGatModel1
from transformers.models.longformer.convert_longformer_original_pytorch_lightning_to_pytorch import LightningModel
mymodel = GcnGatModel1(100, 10, config)
print(type(mymodel))
model = LightningModel(mymodel)
print(type(model))

In [ ]:
from typing import Tuple


def sample_func() -> Tuple[int, float]:
    return 1, 2.5

In [ ]:
targets = sample_func

In [ ]:
type(targets)

In [ ]:
from Scripts.Utils.enums import Optimizer

type(Optimizer.ADAM)

In [ ]:
def sample_func():
    return True

In [ ]:
from typing import Callable
import types

In [ ]:
aa: Callable = None
type(aa)